In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential ,load_model
from tensorflow.keras.layers import Dense, Input,MaxPooling2D,Conv2D,Flatten
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample # downsample the dataset
from sklearn.model_selection import train_test_split # split data into training and testing sets
from sklearn.svm import SVC # this will make a support vector machine for classificaiton
from sklearn.model_selection import GridSearchCV #this will do cross validation
from sklearn.metrics import confusion_matrix # this creates a confusion matrix
from sklearn.decomposition import PCA # to perform PCA to plot the data
from sklearn.metrics import precision_score, accuracy_score, recall_score
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
from PIL import Image
import random
import glob
import os
import cv2
import pandas as pd

In [2]:
# pre processing

# Specify the directory path
directory_path = './resizedImage256'

# Get all files in the directory
files = glob.glob(directory_path + '/*')
file_types = set()
img_height = 256 
img_width = 256
Xdata = []
Ydata = []
for a in files:
    #file_name, file_extension = os.path.splitext(a)
    #file_types.add(file_extension)
    image = cv2.imread(a)
    #image = cv2.resize(image, (img_width,img_height))
    #Image.fromarray(image).save('./resizedImage256/'+file_name.replace("./data\\", "")+file_extension)
    #print('./data/resized256/'+file_name+file_extension)
    #cv2.imwrite('./resizedImage256/'+file_name+file_extension, image)
    image = image / 255.0 # for normallization and having better accuracy
    Xdata.append(image)
    Ydata.append(int("happy" in a))
del files    